# 1. Projeto, Data Understanding & Setup

#### 1.1 - Definição do Projeto & Motivação

> No dia 05/05/2023 a Organização Mundial da Saúde (OMS) declarou o fim da Emergência de Saúde Pública de Importância Internacional referente à COVID-19. [Fonte :OMS](https://www.paho.org/pt/noticias/5-5-2023-oms-declara-fim-da-emergencia-saude-publica-importancia-internacional-referente). 
>
> Este anúncio traz alivio e fim a 3 longos anos, marcados por muita incerteza, perdas e mudanças talvez incomensuráveis na sociedade e no relacionamento humano.
>
> `Este projeto visa analisar os dados da COVID 19 no Brasil e extrair insights de como a mesma se comportou no país nos últimos 3 anos.`

#### 1.2 - Sobre os Dados

> Neste projeto, irei utilizar os dados coletados e compilados pelo Centro de Ciência de Sistemas e Engenharia da universidade americana **John Hopkins** ([link](https://www.jhu.edu)). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.). O site do projeto pode ser acessado neste [link](https://systems.jhu.edu/research/public-health/ncov/) enquanto os dados, neste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país; 
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

#### 1.3 - Sobre o Autor

> Olá!
>
> Me chamo Leonardo, sou estudante do último ano de Engenharia Mecânica que tem como objetivo profissional migrar para a área de dados. Durante os anos de faculdade, trabalhei durante 2 anos com pesquisa na área aeroespacial (lançadores de satélite) no Instituto de Aeronáutica e Espaço, em São José dos Campos (IAE-DCTA). Lá tive meu primeiro contato com a programação e com dados (maioria proveniente de sensores acoplados aos foguetes) e, unindo essa experiencia com meu interesse pelas aulas de estatística durante a graduação, acabei conhecendo a área de ciencia de dados e me apaixonando.

#### 1.4 - Setup

In [10]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from typing import Iterator
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# 2. Extração dos Dados - Casos

## 2.1 Realizando a Extração

<font color = red><b>Atenção:</b></font> Esta seção não precisa ser executada. Para rodar o notebook, realizar o carregamento do arquivo utilizando o Pandas através do arquivo `filtered-raw-data.pkl`.

> Os dados estão salvos no formato `.csv`, onde cada arquivo corresponde a um dia do ano.

* Criando um `Generator` para criar nosso `Iterator`

In [ ]:
def date_range(data_inicial: datetime, data_final: datetime) -> Iterator:

    """
    Generator.
    Essa função recebe duas datas como input (uma data inicial e outra final), criando um Iterator
    com todas as datas entre elas. 
    A granularidade é em dias.

    Exemplo:\n\n
    >> Input\n
    data_inicial = 01/01/2021\n
    data_final = 05/01/2021\n
    date_range(data_inicial, data_final)\n
    \n
    >> Output\n
    Iterator contendo os valores: 01/01/2021, 02/01/2021, 03/01/2021, 04/01/2021 e 05/01/2021.
    """
    # Calculando a quantidade de dias
    days_range = (data_final - data_inicial).days

    # Criando o Iterator
    for lag in range(days_range):
        yield data_inicial + timedelta(lag)

* Setando a data inicial e final

In [ ]:
data_inicial = datetime(2021, 1, 1)
data_final = datetime(2023, 1, 1)

* Extraindo os dados

In [ ]:
# Lista para salvar as datas que não foram carregadas
failed = list()

# Objeto para criar o data frame Pandas
df = pd.DataFrame()

# Iterando sob todas as datas entre o intervalo que queremos considerar
for date in date_range(data_inicial = data_inicial, data_final = data_final):
    # Convertendo a data para string
    date_str = date.strftime('%m-%d-%Y')

    # Acessando o .csv através da url
    source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

    
    try: # Tenta acessar o arquivo referente à data da iteração atual

        case = pd.read_csv(source_url, sep=',')

    except: # Caso dê algum erro (possivelmente o arquivo não exista)

        failed.append(date_str)
        continue

    else: # Caso consiga ler com sucesso o arquivo, adicionamos ao data frame

        # Removendo as colunas que não iremos utilizar
        case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
        # Filtrando apenas os dados do Brasil
        case = case.query('Country_Region == "Brazil"').reset_index(drop= True)
        # Convertendo a coluna de Data
        case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))
        # Inserindo os dados dessa iteração ao data frame
        df = pd.concat([df, case], axis= 0, ignore_index = True)

* Uma vez que o dataframe extraído é pequeno (e o processo de extração é relativamente demorado), irei exporta-lo em `.csv` caso haja necessidade de resetar o kernel.

In [ ]:
df.to_pickle('./filtered-raw-data.pkl', protocol= 5)

## 2.2 Checando os Resultados da Extração

* Carregando os dados

In [ ]:
df = pd.read_pickle('./filtered-raw-data.pkl')

* Checando o resultado da extração dos dados (shape, dtypes e nulos)

In [ ]:
df.head(5)

In [ ]:
# A quantidade de linhas faz sentido (27 estados x 730 dias)
df.shape

In [ ]:
# Não temos dados nulos e os data types
df.info()

# 3. Data Wrangling - Casos

* Ajustando o nome das colunas

In [ ]:
# Mudando o nome das colunas Province_State e Country_Region
df = df.rename(
    columns=
    {
        'Province_State' : 'state',
        'Country_Region' : 'country'
    }
)

In [ ]:
# Passando o nome de todas as colunas para minúsculo
df.columns = [col.lower() for col in df.columns]

* Ajustando o nome dos estados

(Importante para o Looker Studio reconhece-los como uma localização geográfica)

In [ ]:
# Criando um mapper com os nomes
mapper = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Maranhao' : 'Maranhão',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

# Aplicando o mapper na variável 'state'
df['state'] = df['state'].apply(lambda state: mapper.get(state) if state in mapper.keys() else state)

# Mostrando o resultado da operação
df['state'].unique()

* Criando chaves temporais: Ano-Mês

In [ ]:
df['month'] = df['date'].map(lambda date: date.strftime('%Y-%m'))

* Criando chaves temporais: Ano

In [ ]:
df['year'] = df['date'].map(lambda date: date.strftime('%Y'))

* Estimando a população de cada Estado.

Sabemos que a incidencia (`incident rate`) é uma estatística que mede a ocorrência de novos casos baseado na população total que está exposta ao risco. Como sabemos que esta taxa, neste estudo, está baseada em **100.000** habitantes, podemos encontrar esses valores da seguinte maneira:

$$incident\hspace{1mm}rate = \dfrac{100.000 \times confirmed}{population}$$

Analogamente, conseguimos estimar a população total da seguinte maneira:


$$population = \dfrac{100.000 \times confirmed}{incident\hspace{1mm}rate}$$

In [ ]:
df['population'] = round(100000 * (df['confirmed'] / df['incident_rate']))

In [ ]:
# Removendo a coluna incident_rate, já que ela nos dá pouca informação
df = df.drop(['incident_rate'], axis= 1)

* Trabalhando com número de casos confirmados:

1. Casos diários

2. Média Móvel (7 dias)

3. Estabilidade (14 dias)

4. Definindo a tendencia da doença (crescente, decrescente ou estável) com base na estabilidade

In [ ]:
def tendencia(rate: float) -> str:
    
    """
    Função criada para determinar a tendencia de desenvolvimento da doença.\n
    Caso a taxa seja maior que 1.15, a contaminação é considerada crescente.\n
    Caso a taxa seja menor que 0.75, a contaminação é considerada decrescente.\n
    Para outras situações, considera-se a mesma como estável.
    \n
    >> Input\n
    rate (float): Razão entre a média móvel atual com a média móvel anterior.
    \n
    >> Output\n
    status(str): Tendencia de desenvolvimento da doença.
    """
    if np.isnan(rate):
        return np.NaN
    
    if rate <= 0.75:
        status = 'downward'
    elif rate <= 1.15:
        status = 'upward'
    else:
        status = 'stable'
    
    return status

In [ ]:
df_cases_wrangled = pd.DataFrame()

for state in df['state'].unique():
    
    df_queried = (
        df
        .query('state == @state') # Filtra pelo estado
        .reset_index(drop= True) # Arruma a numeração das linhas
        .sort_values(by= 'date') # Ordena pela data
        .copy() # Cria uma cópia, evitando que o original seja alterado
    )
    
    # Calculando o número de casos/mortes por dia
    df_queried['confirmed_1d'] = df_queried['confirmed'].diff(periods= 1)
    df_queried['deaths_1d'] = df_queried['deaths'].diff(periods = 1)
    
    # Calculando a média móvel de casos/mortes dos últimos 7 dias
    df_queried['confirmed_moving_avg_7d'] = np.ceil(df_queried['confirmed_1d'].rolling(window= 7).mean())
    df_queried['deaths_moving_avg_7d'] = np.ceil(df_queried['deaths_1d'].rolling(window= 7).mean())
    
    # Calculando a taxa de desenvolvimento da doença (comparando a média móvel atual com a anterior)
    df_queried['confirmed_moving_avg_7d_rate_14'] = df_queried['confirmed_moving_avg_7d']/df_queried['confirmed_moving_avg_7d'].shift(periods= 14)
    df_queried['deaths_moving_avg_7d_rate_14'] = df_queried['deaths_moving_avg_7d']/df_queried['deaths_moving_avg_7d'].shift(periods= 14)
    
    # Adicionando a coluna com o indicador do desenvolvimento da doença
    df_queried['confirmed_trend'] = df_queried['confirmed_moving_avg_7d_rate_14'].apply(tendencia)
    df_queried['deaths_trend'] = df_queried['deaths_moving_avg_7d_rate_14'].apply(tendencia)
    
    df_cases_wrangled = pd.concat([df_cases_wrangled, df_queried], axis= 0, ignore_index= True)

* Type casting as colunas do data frame final

In [ ]:
# Consumo de memória: 2.4mb
df_cases_wrangled.info()

In [ ]:
for col in df_cases_wrangled.select_dtypes('number').columns:

    try:
        df_cases_wrangled[col] = df_cases_wrangled[col].astype('Int32')
    except:
        df_cases_wrangled[col] = df_cases_wrangled[col].astype('float16')

In [ ]:
# Consumo de memória: 1.8mb -- 25% menos memória utilizada após o type casting
df_cases_wrangled.info()

> Carregamento

* Com os dados já manipulados, vamos carrega-lo em um formato que o Google Data Studio consiga trabalhar.

In [ ]:
df_cases_wrangled.to_csv('./covid-cases-wrangled', sep= ',', index= False)

# 4. Extração dos Dados - Vacinação

* Os dados de vacinação estão compilados em um único arquivo `.csv`

In [15]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep= ',', parse_dates=[3], infer_datetime_format= True, low_memory= False)

In [16]:
vaccines.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [17]:
vaccines.shape

(311568, 67)

* Filtrando os dados do Brasil

In [18]:
vaccines_br = vaccines.query('location == "Brazil"').reset_index(drop= True).copy(deep= True)
vaccines_br.shape

(1231, 67)

* Selecionando apenas as colunas relevantes

In [20]:
vaccines_br = vaccines_br[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [27]:
vaccines_br.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-03
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-04
2,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-05
3,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-06
4,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-07


# 5. Data Wrangling - Vacinação

* Filtrando as datas (Estamos interessados apenas no mesmo intervalo do data frame dos casos)

In [29]:
data_inicial = '2021-1-1'
data_final = '2023-1-1'

In [33]:
vaccines_br = vaccines_br.query('date>=@data_inicial & date<=@data_final').reset_index(drop= True)
vaccines_br

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2021-01-01
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2021-01-02
2,Brazil,215313504.0,NaN,NaN,NaN,NaN,2021-01-03
3,Brazil,215313504.0,NaN,NaN,NaN,NaN,2021-01-04
4,Brazil,215313504.0,NaN,NaN,NaN,NaN,2021-01-05
...,...,...,...,...,...,...,...
726,Brazil,215313504.0,NaN,NaN,NaN,NaN,2022-12-28
727,Brazil,215313504.0,480331769.0,188552661.0,174886102.0,122629436.0,2022-12-29
728,Brazil,215313504.0,480332769.0,188553047.0,174886846.0,NaN,2022-12-30
729,Brazil,215313504.0,480333910.0,188553932.0,174887915.0,122629436.0,2022-12-31


* Verificando e Tratando os Nulos

In [35]:
vaccines_br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 731 non-null    object        
 1   population               731 non-null    float64       
 2   total_vaccinations       637 non-null    float64       
 3   people_vaccinated        633 non-null    float64       
 4   people_fully_vaccinated  617 non-null    float64       
 5   total_boosters           404 non-null    float64       
 6   date                     731 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 40.1+ KB


In [37]:
# Iremos preencher com o valor válido mais próximo
vaccines_br = vaccines_br.fillna(method= 'ffill')

In [38]:
vaccines_br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   location                 731 non-null    object        
 1   population               731 non-null    float64       
 2   total_vaccinations       715 non-null    float64       
 3   people_vaccinated        715 non-null    float64       
 4   people_fully_vaccinated  696 non-null    float64       
 5   total_boosters           487 non-null    float64       
 6   date                     731 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 40.1+ KB


* Criando chaves temporais (mês-ano e ano)

In [39]:
vaccines_br['month'] = vaccines_br['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines_br['year'] = vaccines_br['date'].apply(lambda date: date.strftime('%Y'))

* Renomeando colunas

In [57]:
vaccines_br = vaccines_br.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

* Criando dados relativos

In [58]:
vaccines_br['one_shot_perc'] = round(vaccines_br['one_shot'] / vaccines_br['population'], 2)
vaccines_br['two_shots_perc'] = round(vaccines_br['two_shots'] / vaccines_br['population'], 2)
vaccines_br['three_shots_perc'] = round(vaccines_br['three_shots'] / vaccines_br['population'], 2)

* Type Casting

In [59]:
vaccines_br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   country           731 non-null    object        
 1   population        731 non-null    float64       
 2   total             715 non-null    float64       
 3   one_shot          715 non-null    float64       
 4   two_shots         696 non-null    float64       
 5   three_shots       487 non-null    float64       
 6   date              731 non-null    datetime64[ns]
 7   month             731 non-null    object        
 8   year              731 non-null    object        
 9   one_shot_perc     715 non-null    float64       
 10  two_shots_perc    696 non-null    float64       
 11  three_shots_perc  487 non-null    float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 68.7+ KB


In [60]:
vaccines_br['population'] = vaccines_br['population'].astype('Int32')
vaccines_br['total'] = vaccines_br['total'].astype('Int32')
vaccines_br['one_shot'] = vaccines_br['one_shot'].astype('Int32')
vaccines_br['two_shots'] = vaccines_br['two_shots'].astype('Int32')
vaccines_br['three_shots'] = vaccines_br['three_shots'].astype('Int32')

In [61]:
vaccines_br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   country           731 non-null    object        
 1   population        731 non-null    Int32         
 2   total             715 non-null    Int32         
 3   one_shot          715 non-null    Int32         
 4   two_shots         696 non-null    Int32         
 5   three_shots       487 non-null    Int32         
 6   date              731 non-null    datetime64[ns]
 7   month             731 non-null    object        
 8   year              731 non-null    object        
 9   one_shot_perc     715 non-null    float64       
 10  two_shots_perc    696 non-null    float64       
 11  three_shots_perc  487 non-null    float64       
dtypes: Int32(5), datetime64[ns](1), float64(3), object(3)
memory usage: 57.9+ KB


In [62]:
vaccines_br

,country,population,total,one_shot,two_shots,three_shots,date,month,year,one_shot_perc,two_shots_perc,three_shots_perc
0,Brazil,215313504,<NA>,<NA>,<NA>,<NA>,2021-01-01,2021-01,2021,NaN,NaN,NaN
1,Brazil,215313504,<NA>,<NA>,<NA>,<NA>,2021-01-02,2021-01,2021,NaN,NaN,NaN
2,Brazil,215313504,<NA>,<NA>,<NA>,<NA>,2021-01-03,2021-01,2021,NaN,NaN,NaN
3,Brazil,215313504,<NA>,<NA>,<NA>,<NA>,2021-01-04,2021-01,2021,NaN,NaN,NaN
4,Brazil,215313504,<NA>,<NA>,<NA>,<NA>,2021-01-05,2021-01,2021,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
726,Brazil,215313504,480310839,188549744,174881292,122616211,2022-12-28,2022-12,2022,0.88,0.81,0.57
727,Brazil,215313504,480331769,188552661,174886102,122629436,2022-12-29,2022-12,2022,0.88,0.81,0.57
728,Brazil,215313504,480332769,188553047,174886846,122629436,2022-12-30,2022-12,2022,0.88,0.81,0.57
729,Brazil,215313504,480333910,188553932,174887915,122629436,2022-12-31,2022-12,2022,0.88,0.81,0.57


> Carregamento

* Com os dados já manipulados, vamos carrega-lo em um formato que o Google Data Studio consiga trabalhar.

In [63]:
vaccines_br.to_csv('./vaccines-wrangled', sep= ',', index= False)